In [0]:
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/iemocap/data.zip"

In [8]:
!git clone "https://github.com/convman/Multimodal.git"

Cloning into 'Multimodal'...
remote: Enumerating objects: 350, done.
remote: Total 350 (delta 0), reused 0 (delta 0), pack-reused 350
Receiving objects: 100% (350/350), 892.26 MiB | 41.29 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Checking out files: 100% (148/148), done.


In [0]:
!unzip -P "hello_iemocap_2008" data.zip

In [5]:
import pickle

Train_video = pickle.load(open("data/facet_train.p",'rb'),encoding='latin1')
Test_video = pickle.load(open("data/facet_test.p",'rb'),encoding='latin1')
Valid_video = pickle.load(open("data/facet_valid.p",'rb'),encoding='latin1')

Train_text = pickle.load(open("data/text_train.p",'rb'),encoding='latin1')
Test_text = pickle.load(open("data/text_test.p",'rb'),encoding='latin1')
Valid_text = pickle.load(open("data/text_valid.p",'rb'),encoding='latin1')

Train_audio = pickle.load(open("data/covarep_train.p",'rb'),encoding='latin1')
Test_audio = pickle.load(open("data/covarep_test.p",'rb'),encoding='latin1')
Valid_audio = pickle.load(open("data/covarep_valid.p",'rb'),encoding='latin1')

Y_train =  pickle.load(open("data/y_train.p",'rb'),encoding='latin1')
Y_test =  pickle.load(open("data/y_test.p",'rb'),encoding='latin1')
Y_valid =  pickle.load(open("data/y_valid.p",'rb'),encoding='latin1')

Y_train = Y_train[:,1:-1]  # selecting only emotion label and ignoring others and unknown
Y_test = Y_test[:,1:-1]    # regarding 11 features:
Y_valid = Y_valid[:,1:-1]  # Other Anger Excited Fear Sad Surprised Frustrated Happy Neutral Disgust (unknown)

print(Train_video.shape,Test_video.shape,Valid_video.shape)
print(Train_text.shape,Test_text.shape,Valid_text.shape)
print(Train_audio.shape,Test_audio.shape,Valid_audio.shape)



Y_train = Y_train[:,[0,1,3,5,6,7]]
Y_test = Y_test[:,[0,1,3,5,6,7]]
Y_valid = Y_valid[:,[0,1,3,5,6,7]]
target_names = ['Anger','Excited','Sad','Frustrated','Happy','Neutral']
map = {0:"Anger",1:"Excited",2:"Fear",3:"Sad",4:"Surprised",5:"Frustrated",6:"Happy",7:"Neutral",8:"Disgust"}
print(Y_train.shape,Y_test.shape,Y_valid.shape)

(6373, 21, 35) (1807, 21, 35) (1775, 21, 35)
(6373, 21, 300) (1807, 21, 300) (1775, 21, 300)
(6373, 21, 74) (1807, 21, 74) (1775, 21, 74)
(6373, 6) (1807, 6) (1775, 6)


In [0]:
import pandas as pd
import numpy as np
import h5py
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.utils.multiclass import unique_labels
import keras
import tensorflow as tf
from time import time
import scipy
from scipy.io import savemat
from keras import layers
from google.colab import files	
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import *
from keras.callbacks import TensorBoard
from keras import callbacks
from keras.callbacks import *
%matplotlib inline

In [7]:
Train_concat = np.concatenate((Train_audio,Train_text,Train_video),axis=-1)
Test_concat = np.concatenate((Test_audio,Test_text,Test_video),axis=-1)
Valid_concat = np.concatenate((Valid_audio,Valid_text,Valid_video),axis=-1)

print(Train_concat.shape)
print(Test_concat.shape)
print(Valid_concat.shape)

(6373, 21, 409)
(1807, 21, 409)
(1775, 21, 409)


In [0]:
model = load_model("Multimodal/iemocap sdk/multimodal/v2 - late fusion/weights/late_fusion.h5")

In [30]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           (None, 21, 74)       0                                            
__________________________________________________________________________________________________
input_77 (InputLayer)           (None, 21, 35)       0                                            
__________________________________________________________________________________________________
input_78 (InputLayer)           (None, 21, 300)      0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 64)           194688      input_76[0][0]                   
__________________________________________________________________________________________________
sequential

In [31]:
m = Model(model.input,model.get_layer("dense_150").output)
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           (None, 21, 74)       0                                            
__________________________________________________________________________________________________
input_77 (InputLayer)           (None, 21, 35)       0                                            
__________________________________________________________________________________________________
input_78 (InputLayer)           (None, 21, 300)      0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 64)           194688      input_76[0][0]                   
__________________________________________________________________________________________________
sequential

In [32]:
Dtrain = m.predict([Train_audio,Train_video,Train_text])
Dtest = m.predict([Test_audio,Test_video,Test_text])
print(Dtrain.shape,Dtest.shape)

savemat("Dtrain.mat",{'val':Dtrain})
savemat("Dtest.mat",{'val':Dtest})

files.download("Dtrain.mat")
files.download("Dtest.mat")

(6373, 64) (1807, 64)
